### Delete this later
NOTE TBD: means notes for us, to be deleted

# STINTSY Machine Project
#### S14 Group X
- Aldecoa, Renzel Raphael
- Bautista, Carlo Gabriel
- Canicon, Jan Ambro

# CS:GO Cheating Dataset
The [dataset](https://www.kaggle.com/datasets/emstatsl/csgo-cheating-dataset) used features gameplay input from players, segregated into non-cheating (henceforth "legit") and cheating players. The main task with the dataset is to categorize players as "cheating" based on their game input. There are a total of `12 000` samples in the dataset, divided into two groups of `10 000` legit players and `2 000` cheating players.

The dataset has a shape of `[X, 30, 192, 5]`, with the dimensions as follows:
- `X` — `player` the number of players in the subset (`10 000` legit, `2 000` cheater)
- `30` — `engagement` specifies the encounter with other players
- `192` — `game_tick` in-game time increment holding player vectors and other game info
- `5` — `player_input` input sent by players such as change in aim yaw and pitch

The player input dimension of the dataset can be further explained as follows:
- `AttackerDeltaYaw`, `AttackerDeltaPitch`
    - Change in the direction of the attacking player’s aim
- `CrosshairToVictimYaw`, `CrosshairToVictimPitch`
    - Distance remaining between the attacker’s crosshair and target’s hitbox
- `Firing`
    - Whether the attacker is firing or not

Additionally note that yaw refers to the left-right movement axis, and pitch refers to up-down movement axis. These are scaled in the 360-degree measurement, with a range of `[-180, 180]`.

TODO: Explain features more

# Dependencies
NOTE TBD: The `pip install` stuff

# Library Imports

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Loading the dataset

In [4]:
legit_data = np.load('legit.npy')
cheaters_data = np.load('cheaters.npy')

print('Legit:', legit_data.shape)
print('Cheaters:', cheaters_data.shape)

Legit: (10000, 30, 192, 5)
Cheaters: (2000, 30, 192, 5)


# Preprocessing
TODO: Possibly balance the ratio of legit and cheaters?

The dataset is stored in a format of `[player, engagement, game_tick, player_input]`, and we can further flatten it by combining `engagement` and `game_tick` as `tick`, because these are all []. However, we cannot further flatten `player_input` with `tick` because these are the features, and flattening would scramble them. Thus, we will be training and testing the model with a 3-dimensional dataset later on.

After flattening the dimensions, we can also combine the two datasets.

In [5]:
# Combines the dimensions engagement and game_tick
legit_data = legit_data.reshape(10000, 5760, 5)
cheaters_data = cheaters_data.reshape(2000, 5760, 5)

# Combine legit and cheater datasets
X_players = np.vstack([legit_data, cheaters_data])
print('Training data shape:', X_players.shape)

Training data shape: (12000, 5760, 5)


Since we append the cheaters to the end of the list of legit players, we can easily create a list that contains their classes by using the length of each list. The classes will be:
- `0` — `Legit` player
- `1` — `Cheating` player

In [6]:
# Create truth values for combined list
y_players = np.append(
        np.zeros(legit_data.shape[0], dtype=int),
        np.full(cheaters_data.shape[0], 1, dtype=int)
)
print('Ground truth values shape:', y_players.shape)

Ground truth values shape: (12000,)


In order to add more randomness for the splitting of the train and test sets later on, we shuffle this combined list. To do so while keeping the correspondence of the rows from both lists, we shuffle the indices instead and assign them to both lists.

In [9]:
# https://stackoverflow.com/questions/23289547/shuffle-two-list-at-once-with-same-order
# Generate shuffled indices
indices = np.arange(X_players.shape[0])
np.random.shuffle(indices)

# Assign shuffled indices to both lists
X_players = X_players[indices]
y_players = y_players[indices]

In [ ]:
legit

In [ ]:
legit.describe()

In [ ]:
cheater.describe()

In [ ]:
fig = plt.figure(figsize=(15,15))

ax1 = fig.add_subplot(411)
plt.plot(legit['AttackerDeltaYaw'])
plt.plot(cheater['AttackerDeltaYaw'])
ax1.set_title('AttackerDeltaYaw')

ax2 = fig.add_subplot(412)
plt.plot(legit['AttackerDeltaPitch'])
plt.plot(cheater['AttackerDeltaPitch'])
ax2.set_title('AttackerDeltaPitch')

ax3 = fig.add_subplot(413)
plt.plot(legit['CrosshairToVictimYaw'])
plt.plot(cheater['CrosshairToVictimYaw'])
ax3.set_title('CrosshairToVictimYaw')

ax4 = fig.add_subplot(414)
plt.plot(legit['CrosshairToVictimPitch'])
plt.plot(cheater['CrosshairToVictimPitch'])
ax4.set_title('CrosshairToVictimPitch')

In [ ]:
cols = ['AttackerDeltaYaw', 'AttackerDeltaPitch', 'CrosshairToVictimYaw', 'CrosshairToVictimPitch', 'Firing']
rows = ['Tick ' + str(i) for i in range(1, 193)]

legit_df = pd.DataFrame(legit_data[0][0], index=rows, columns=cols)
cheaters_df = pd.DataFrame(cheaters_data[0][0], index=rows, columns=cols)

# Test (Based on NB5)